In [2]:
import duckdb
duckdb.sql("ATTACH 'dbname=mimic4 user=szu004' AS db (TYPE POSTGRES, READ_ONLY)")
duckdb.sql("USE db")

In [3]:
# ALL ICU stays (paper reports 765,40)
duckdb.sql("SELECT COUNT(*) as icu_stay_count FROM mimiciv_derived.icustay_detail")

┌────────────────┐
│ icu_stay_count │
│     int64      │
├────────────────┤
│          73181 │
└────────────────┘

In [4]:
# ALL Unique Patients (paper reports 53,150)
duckdb.sql("SELECT COUNT(DISTINCT subject_id) as patient_count FROM mimiciv_derived.icustay_detail")

┌───────────────┐
│ patient_count │
│     int64     │
├───────────────┤
│         50920 │
└───────────────┘

In [5]:
# Patients with documented enthicity
duckdb.sql("""
WITH patients AS (
    SELECT subject_id,
    CASE
        WHEN race LIKE 'ASIAN%' THEN 'ASIAN'
        WHEN race LIKE 'BLACK%' THEN 'BLACK'
        WHEN race LIKE 'WHITE%' THEN 'WHITE'
        WHEN race LIKE 'HISPANIC%' THEN 'HISPANIC'
        ELSE 'UNKNOWN'
        END AS race_category
    FROM mimiciv_derived.icustay_detail WHERE first_icu_stay AND first_hosp_stay)
SELECT race_category, COUNT(*) as patient_count FROM patients WHERE RACE_CATEGORY != 'UNKNOWN' GROUP BY race_category ORDER BY race_category
""")

┌───────────────┬───────────────┐
│ race_category │ patient_count │
│    varchar    │     int64     │
├───────────────┼───────────────┤
│ ASIAN         │          1496 │
│ BLACK         │          4640 │
│ HISPANIC      │          1735 │
│ WHITE         │         34204 │
└───────────────┴───────────────┘

In [22]:
patient_with_race = duckdb.sql("""
WITH patients AS (
    SELECT *,
    CASE
        WHEN race LIKE 'ASIAN%' THEN 'ASIAN'
        WHEN race LIKE 'BLACK%' THEN 'BLACK'
        WHEN race LIKE 'WHITE%' THEN 'WHITE'
        WHEN race LIKE 'HISPANIC%' THEN 'HISPANIC'
        ELSE 'UNKNOWN'
        END AS race_category
    FROM mimiciv_derived.icustay_detail WHERE first_icu_stay AND first_hosp_stay)
SELECT subject_id, stay_id, icu_intime, icu_outtime, race_category  FROM patients WHERE race_category != 'UNKNOWN'
""")
patient_with_race

┌────────────┬──────────┬─────────────────────┬─────────────────────┬───────────────┐
│ subject_id │ stay_id  │     icu_intime      │     icu_outtime     │ race_category │
│   int32    │  int32   │      timestamp      │      timestamp      │    varchar    │
├────────────┼──────────┼─────────────────────┼─────────────────────┼───────────────┤
│   10000032 │ 39553978 │ 2180-07-23 14:00:00 │ 2180-07-23 23:50:47 │ WHITE         │
│   10000980 │ 39765666 │ 2189-06-27 08:42:00 │ 2189-06-27 20:38:27 │ BLACK         │
│   10001217 │ 37067082 │ 2157-11-20 19:18:02 │ 2157-11-21 22:08:00 │ WHITE         │
│   10001725 │ 31205490 │ 2110-04-11 15:52:22 │ 2110-04-12 23:59:56 │ WHITE         │
│   10001884 │ 37510196 │ 2131-01-11 04:20:05 │ 2131-01-20 08:27:30 │ BLACK         │
│   10002155 │ 33685454 │ 2129-08-04 12:45:00 │ 2129-08-10 17:02:38 │ WHITE         │
│   10002348 │ 32610785 │ 2112-11-30 23:24:00 │ 2112-12-10 18:25:13 │ WHITE         │
│   10002428 │ 33987268 │ 2156-04-12 16:24:18 │ 2156-0

In [23]:
# Make sure that patients are unique
duckdb.sql("""
SELECT subject_id, COUNT(*) as patient_count FROM patient_with_race GROUP BY subject_id HAVING COUNT(*) > 1
""")

┌────────────┬───────────────┐
│ subject_id │ patient_count │
│   int32    │     int64     │
├────────────┴───────────────┤
│           0 rows           │
└────────────────────────────┘

In [24]:
# Make sure that patients are unique
duckdb.sql("""
SELECT COUNT(*) FROM patient_with_race
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        42075 │
└──────────────┘

In [38]:
# Compute patients index period and select valid parient
stays_with_interventions = duckdb.sql("""
WITH first_vent_intervention AS (
    SELECT stay_id, MIN(starttime) as  starttime FROM mimiciv_derived.ventilation
    WHERE ventilation_status NOT in ('None', 'SupplementalOxygen') GROUP BY stay_id)
SELECT pwr.*,v.* FROM patient_with_race AS pwr
LEFT OUTER JOIN first_vent_intervention AS v ON pwr.stay_id = v.stay_id
""")
patient_with_index_period=duckdb.sql("""
WITH patient_with_period AS (SELECT subject_id, stay_id, race_category, 
icu_intime AS ip_starttime, 
LEAST(icu_outtime, starttime, icu_intime + interval '5 days') AS ip_endtime FROM stays_with_interventions)
SELECT *,(ip_endtime-ip_starttime) AS ip_duration FROM patient_with_period 
WHERE (ip_endtime-ip_starttime) >= interval '12 hours'
""")
patient_with_index_period

┌────────────┬──────────┬───────────────┬─────────────────────┬─────────────────────┬─────────────────┐
│ subject_id │ stay_id  │ race_category │    ip_starttime     │     ip_endtime      │   ip_duration   │
│   int32    │  int32   │    varchar    │      timestamp      │      timestamp      │    interval     │
├────────────┼──────────┼───────────────┼─────────────────────┼─────────────────────┼─────────────────┤
│   10001217 │ 37067082 │ WHITE         │ 2157-11-20 19:18:02 │ 2157-11-21 22:08:00 │ 1 day 02:49:58  │
│   10001725 │ 31205490 │ WHITE         │ 2110-04-11 15:52:22 │ 2110-04-12 23:59:56 │ 1 day 08:07:34  │
│   10002155 │ 33685454 │ WHITE         │ 2129-08-04 12:45:00 │ 2129-08-09 12:45:00 │ 5 days          │
│   10002348 │ 32610785 │ WHITE         │ 2112-11-30 23:24:00 │ 2112-12-05 23:24:00 │ 5 days          │
│   10002428 │ 33987268 │ WHITE         │ 2156-04-12 16:24:18 │ 2156-04-17 15:57:08 │ 4 days 23:32:50 │
│   10002443 │ 35044219 │ WHITE         │ 2183-10-18 00:47:00 │ 

In [42]:
duckdb.sql("""
SELECT COUNT( subject_id) FROM patient_with_index_period
""")

┌───────────────────┐
│ count(subject_id) │
│       int64       │
├───────────────────┤
│             26521 │
└───────────────────┘

In [53]:
patients_with_flow = duckdb.sql("""
SELECT pwr.subject_id, pwr.race_category,od.charttime as flow_time, od.o2_flow FROM patient_with_index_period AS pwr
JOIN mimiciv_derived.oxygen_delivery AS od ON  pwr.stay_id = od.stay_id
WHERE od.o2_delivery_device_1 = 'Nasal cannula' AND od.charttime BETWEEN pwr.ip_starttime AND pwr.ip_endtime AND od.o2_flow IS NOT NULL
ORDER BY pwr.subject_id, od.charttime
""")
patients_with_flow

┌────────────┬───────────────┬─────────────────────┬─────────┐
│ subject_id │ race_category │      flow_time      │ o2_flow │
│   int32    │    varchar    │      timestamp      │ double  │
├────────────┼───────────────┼─────────────────────┼─────────┤
│   10001217 │ WHITE         │ 2157-11-20 19:47:00 │     2.0 │
│   10001217 │ WHITE         │ 2157-11-20 22:00:00 │     2.0 │
│   10001217 │ WHITE         │ 2157-11-21 00:00:00 │     2.0 │
│   10001217 │ WHITE         │ 2157-11-21 02:00:00 │     2.0 │
│   10001217 │ WHITE         │ 2157-11-21 08:00:00 │     2.0 │
│   10001217 │ WHITE         │ 2157-11-21 18:00:00 │     3.0 │
│   10001217 │ WHITE         │ 2157-11-21 20:00:00 │     4.0 │
│   10001725 │ WHITE         │ 2110-04-11 16:07:00 │     2.0 │
│   10001725 │ WHITE         │ 2110-04-11 20:00:00 │     2.0 │
│   10001725 │ WHITE         │ 2110-04-12 02:00:00 │     2.0 │
│       ·    │   ·           │          ·          │      ·  │
│       ·    │   ·           │          ·          │   

In [54]:
duckdb.sql("""
SELECT subject_id, COUNT(*) AS flow_count FROM patients_with_flow GROUP BY subject_id ORDER BY flow_count DESC
""")

┌────────────┬────────────┐
│ subject_id │ flow_count │
│   int32    │   int64    │
├────────────┼────────────┤
│   17552487 │         48 │
│   18679942 │         44 │
│   11153132 │         43 │
│   13280145 │         43 │
│   15022555 │         42 │
│   12806413 │         41 │
│   18881392 │         40 │
│   19717191 │         40 │
│   19411951 │         40 │
│   15299156 │         39 │
│       ·    │          · │
│       ·    │          · │
│       ·    │          · │
│   13074505 │          4 │
│   13146404 │          4 │
│   13191155 │          4 │
│   13225747 │          4 │
│   13334510 │          4 │
│   13416306 │          4 │
│   13471402 │          4 │
│   13503771 │          4 │
│   13515390 │          4 │
│   13567435 │          4 │
├────────────┴────────────┤
│ ? rows        2 columns │
└─────────────────────────┘

In [55]:
# Save the data
patients_with_flow.to_csv('../data/patients_with_flow.csv')